In [1]:
import argparse
import os
import sys
import psutil
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
import multiprocessing
from joblib import Parallel, delayed
from orderedset import OrderedSet
import datetime

In [2]:
def print_memory_usage():
    print ("memory log:")
    process = psutil.Process(os.getpid())
    print("%5.2f GB (RSS)" % (process.memory_info().rss / 2**30))
    print("%5.2f GB (VMS)" % (process.memory_info().vms / 2**30))
    print("%5.2f GB (Used)" % (psutil.virtual_memory().used / 2**30))
    print("%5.2f GB (Available)" % (psutil.virtual_memory().available / 2**30))
    print("%5.2f GB (Total)" % (psutil.virtual_memory().total / 2**30))

def distance(data,lat,lng,idx):
    fLat = np.radians(data.LocationLat)
    fLon = np.radians(data.LocationLng)
    sLat = np.radians(lat)
    sLon = np.radians(lng)
    R = 3958.7564 #mi
    #R = 6371000.0 #meters
    #R = 6371.0 #km
    
    dLon = sLon - fLon
    dLat = sLat - fLat
    a = np.sin(dLat/2.0)**2 + (np.cos(fLat) * np.cos(sLat) * np.power(np.sin(dLon/2.0), 2))
    
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    #data['dis2event_'+str(idx)] =  
    return R * c
    #return data

class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)

def applyParallel(pool,data, func, kwargs):
    data_split = np.array_split(data, min(partitions,data.shape[0]))
    data = pd.concat(pool.map(WithExtraArgs(func, **kwargs), data_split))
    return data

def parallelize(data, func,pool):
    data_split = np.array_split(data, partitions)
    df = pd.concat(pool.map(func, data_split))
    return df

def midpoint(collision_df):
    #Convert from degrees to radians.
    lat_rad = np.radians(collision_df.LocationLat)
    lng_rad = np.radians(collision_df.LocationLng)
    #display(lat_rad)
    #Convert lat/lon to Cartesian coordinates
    car_x = np.cos(lat_rad)*np.cos(lng_rad)
    car_y = np.cos(lat_rad)*np.sin(lng_rad)
    car_z = np.sin(lat_rad)
    #display(car_x)
    #Compute weight (not implemented)
    avr_x = car_x.mean()
    avr_y = car_y.mean()
    avr_z = car_z.mean()
    new_lng = np.arctan2 (avr_y,avr_x) 
    Hyp = np.sqrt((avr_x*avr_x) + (avr_y*avr_y))
    new_lat = np.arctan2 (avr_z,Hyp)
    new_lat = np.degrees(new_lat)#np.degrees(new_lat)  **** corrected only swaped
    new_lng = np.degrees(new_lng)#np.degrees(new_lng)
    return new_lat,new_lng
    
def merge(collision_df, long_event):
    col_df = collision_df.append(long_event)
    long_event.StartTime = col_df.StartTime.min()
    long_event.EndTime = col_df.EndTime.max()
    long_event.duration = abs((long_event.EndTime - long_event.StartTime).total_seconds())/60
    sub_traffic = col_df[col_df.Type=='T']
    if not sub_traffic.empty:
        long_event.LocationLat, long_event.LocationLng =  midpoint(sub_traffic)
    TYPE = col_df.Type.sort_values()
    UNI_TYPE = OrderedSet(TYPE)
    long_event.Type = '_'.join(UNI_TYPE)
    
    RT = col_df.RefinedType.sort_values()
    UNI_RT = OrderedSet(RT)
    long_event.RefinedType='_'.join(UNI_RT)
    
    EID = col_df.EventId.sort_values()
    UNI_EID = OrderedSet(EID)
    long_event.EventId = '_'.join(UNI_EID)
    
    return long_event

In [3]:
raduis = 14 # mi

cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want
print ("number of cores "+str(partitions))

number of cores 28


In [4]:

data_set = pd.read_hdf('../../data_set.h5',key='DS_new')
print ("dataset is loaded")

dataset is loaded


In [5]:
max_effective_days = 24*21*60
effective_days = 300
print ("Long event duration is: min: "+str(effective_days)+" max: "+str(max_effective_days))
print ("extract long events")
    #print("Long events are:")
long_events = data_set[(data_set.duration >= effective_days) & (data_set.duration <= max_effective_days)]
long_events = pd.concat([long_events[long_events.Type=='W'],long_events[long_events.Type=='T']])
#display(long_events)
print ("*"*80)

print ("long events type:")
display(long_events.groupby("RefinedType").size().to_frame(name="count"))
long_events.to_hdf('../../long_events.h5',key='LN_300')
#long_events = long_events.sample(2000)

Long event duration is: min: 300 max: 30240
extract long events
********************************************************************************
long events type:


,count
RefinedType,
Accident,226
Broken,49
Congestion,17314
Construction,113984
Event,32144
Incident,13099
Lane,253
Other,440
cold,12083


/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [6]:
def merge_events(data):
    data.tolist()
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    print("process ",id," started")
    final_DF=pd.DataFrame()
    for state in data:
        processed_events=[]
        drop_indices=[]
        long_events_ = long_events[long_events.State==state]
        long_events_.sort_values(by=['duration'],inplace=True,ascending=False)
        counter=0
        for idx,long_event in long_events_.iterrows():
            counter+=1
            if counter%1000==0:
                print ("process ", id, counter,"/",len(long_events_), "long event proccesed ",datetime.datetime.now().time())
            if idx not in processed_events:
                
                collision = long_events_[(long_events_.EventId != long_event.EventId )
                            & 
                            (((long_events_.StartTime >= long_event.StartTime) &
                                (long_events_.EndTime <= long_event.EndTime))
                                |
                                ((long_events_.StartTime < long_event.StartTime) &
                                (long_events_.EndTime > long_event.StartTime)) 
                                |
                                ((long_events_.StartTime < long_event.EndTime) &
                                (long_events_.EndTime > long_event.EndTime))
                                |
                                ((long_events_.StartTime < long_event.StartTime) &
                                (long_events_.EndTime > long_event.EndTime)))
                            ]
                
                if long_event.Type == 'W':
                    collision = collision[collision.AirportCode == long_event.AirportCode]
                
                elif long_event.Type == 'T':
                    collison_W = collision[(collision.Type=='W')&(collision.AirportCode == long_event.AirportCode)]
                    collision_T = collision[collision.Type=='T']
                    #collision_T = 
                    
                    #collision_T.loc[:,'dis2event_'+str(idx)] = 10 'dis2event_'+str(idx)
                    collision_T = collision_T.assign(distance=pd.Series(
                        distance(collision_T,long_event.LocationLat,long_event.LocationLng,idx)).values)
                    #distance(collision_T,long_event.LocationLat,long_event.LocationLng,idx)
                    if 'distance' not in collision_T.columns:
                        print ('error')
                        raise()
                    collision_T = collision_T [(collision_T.distance <=raduis)]
                    collision = pd.concat([collison_W,collision_T])
                    
                
                else:
                    print ('error: event type ',long_event.Type)
                    raise()
                
                
                intersection_list = list(set(collision.index) & set(processed_events)) 
                
                collision.drop(intersection_list,inplace=True)
                if not collision.empty:
                    long_events_.loc[idx]= merge(collision,long_event)
                    processed_events.extend(collision.index)
                    processed_events.append(idx)
                    
                    drop_indices.extend(collision.index) 

        long_events_.drop(drop_indices,inplace=True)
        final_DF = final_DF.append(long_events_)
        
    return final_DF

In [7]:
a = long_events.groupby("State").size().to_frame(name="count").reset_index()
display(a)
states=[]
for idx,item in a.iterrows():
    states.append(item.State)

,State,count
0,AL,2050
1,AR,1419
2,AZ,2601
3,CA,24413
4,CO,13439
5,CT,3282
6,DC,1276
7,DE,743
8,FL,10875
9,GA,6601


In [8]:
#long_events_=long_events.copy()
pool = Pool(cores)
data = parallelize(states, merge_events,pool)
pool.close()
pool.join()

process  1  started
process  2  started
process  3  started
process  6  started
process  4  started
process  9  started
process  8  started
process  5  started
process  10  started
process  11  started
process  7  started
process  13  started
process  12  started
process  16  started
process  23  started
process  24  started
process  21  started
process  18  started
process  15  started
process  20  started
process  14  started
process  17  started
process  22  started
process  26  started
process  19  started
process  28  started
process  25  started
process  27  started


/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

process  18 1000 / 4409 long event proccesed  00:35:01.106545


...

process  7 4000 / 4193 long event proccesed  00:56:23.055187
process  21 7000 / 16780 long event proccesed  00:56:27.785092
process  2 8000 / 24413 long event proccesed  00:56:30.005998
process  17 6000 / 31254 long event proccesed  00:56:30.114157
process  15 5000 / 10063 long event proccesed  00:56:35.517732
process  11 1000 / 8592 long event proccesed  00:56:57.475545
process  3 3000 / 3282 long event proccesed  00:57:33.133743
process  19 8000 / 14861 long event proccesed  00:57:41.586672
process  11 2000 / 8592 long event proccesed  00:58:02.683796
process  21 8000 / 16780 long event proccesed  00:58:05.949634
process  17 7000 / 31254 long event proccesed  00:58:19.107758
process  15 6000 / 10063 long event proccesed  00:58:22.110777
process  5 1000 / 6601 long event proccesed  00:58:38.415812
process  11 3000 / 8592 long event proccesed  00:59:13.507255
process  2 9000 / 24413 long event proccesed  00:59:14.209869
process  21 9000 / 16780 long event proccesed  00:59:42.722102
pro

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


process  2 13000 / 24413 long event proccesed  01:07:39.914470
process  19 1000 / 1157 long event proccesed  01:08:25.187922
process  17 12000 / 31254 long event proccesed  01:08:29.278911
process  2 14000 / 24413 long event proccesed  01:08:55.369632
process  2 15000 / 24413 long event proccesed  01:09:45.790152
process  17 13000 / 31254 long event proccesed  01:09:58.730644
process  2 16000 / 24413 long event proccesed  01:10:47.158921
process  17 14000 / 31254 long event proccesed  01:11:14.611301
process  2 17000 / 24413 long event proccesed  01:11:34.515879
process  2 18000 / 24413 long event proccesed  01:12:19.998739
process  2 19000 / 24413 long event proccesed  01:12:58.880444
process  17 15000 / 31254 long event proccesed  01:13:11.904166
process  2 20000 / 24413 long event proccesed  01:13:40.520654
process  2 21000 / 24413 long event proccesed  01:14:16.368636
process  17 16000 / 31254 long event proccesed  01:14:20.410115
process  2 22000 / 24413 long event proccesed  01:1

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


process  17 1000 / 6616 long event proccesed  01:27:28.943256
process  17 2000 / 6616 long event proccesed  01:28:16.332015
process  17 3000 / 6616 long event proccesed  01:29:06.558959
process  17 4000 / 6616 long event proccesed  01:29:47.212361
process  17 5000 / 6616 long event proccesed  01:30:18.140628
process  17 6000 / 6616 long event proccesed  01:30:45.286320


In [9]:
display(data)

,EventId,Type,RefinedType_,StartTime,EndTime,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode,RefinedType,duration
6538970,T-5006887,T,Construction,2018-07-12 09:52:41,2018-07-31 10:22:00,34.189785,-86.526672,0.000000,K8A0,1267.0,State Highway 67,L,Blountsville,Blount,AL,35031.0,Construction,27389.316667
27262,W-36847,W,cold-severe,2018-05-24 16:56:00,2018-06-12 14:56:00,NaN,NaN,NaN,K1M4,NaN,NaN,NaN,Russellville,NaN,AL,NaN,cold,27240.000000
6772201,T-5918907_T-6202981_T-6203021,T,Construction,2018-07-12 09:37:25,2018-07-31 05:00:00,32.600131,-85.432367,67.610001,KAUO,NaN,Columbus Pkwy,R,Opelika,Lee,AL,36804.0,Congestion_Construction,27082.583333
7917957,T-7073427_W-74530_W-74545,T_W,Construction,2018-04-19 14:35:45,2018-05-07 04:59:00,34.824619,-85.785294,4.360000,K4A6,8271.0,AL Highway 117,L,Flat Rock,Jackson,AL,35966.0,Construction_rain,25343.250000
8636738,T-7797145_T-7797146,T,Construction,2018-03-28 15:44:59,2018-04-15 04:59:00,31.542608,-86.220195,6.660000,KLOR,4498.0,Elba Hwy,L,Elba,Crenshaw,AL,36323.0,Construction,25274.016667
6538441,T-5006315_T-5006316,T,Construction,2018-07-16 09:16:38,2018-08-02 05:00:00,34.657398,-86.870995,0.000000,KHSV,26913.0,Powell Rd,R,Madison,Limestone,AL,35756.0,Construction,24223.366667
13716854,T-12913075_T-14134756_T-14233188_T-14325334,T,Construction,2017-08-15 14:02:28,2017-09-01 04:59:00,34.662990,-86.803953,0.000000,KHUA,857.0,Mill Rd,L,Madison,Madison,AL,35758.0,Construction_Other,23936.533333
7192550,T-6342603,T,Construction,2018-05-15 14:16:36,2018-06-01 03:59:00,32.502323,-85.048828,0.000000,KCSG,NaN,Price Rd,R,Phenix City,Russell,AL,36870.0,Construction,23862.400000
7198957,T-6349038_T-7438981_T-7438982_T-7439443_T-7439...,T_W,Construction,2018-05-15 15:26:59,2018-06-01 05:00:00,31.867370,-86.620505,4.100000,KPRN,NaN,Fort Dale Rd,R,Greenville,Butler,AL,36037.0,Construction_rain,23853.016667
7199476,T-6349561_T-7439445_T-7439446_W-1854911,T_W,Construction,2018-05-15 15:19:28,2018-05-31 23:00:00,32.166855,-86.432257,9.870000,KMGM,NaN,I-65 S,R,Hope Hull,Lowndes,AL,36043.0,Construction_fog,23500.533333


In [10]:
data.to_hdf('../../long_events_.h5',key='LN_300')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [11]:
data.groupby('Type').size().sort_values()

Type
T_W     4436
T      55640
W      96291
dtype: int64